<a href="https://colab.research.google.com/github/dwhew/Bitou-bush/blob/main/bitou_read.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

In [ ]:
import pandas as pd
import geopandas as gpd
import fiona


In [ ]:
#use fiona to help read geopandas read the file
# notice the description column is html

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
df = gpd.read_file(r'https://raw.githubusercontent.com/dwhew/Bitou-bush/main/AllBitou20b.kml', driver='KML')
df

In [ ]:
#again showing the html string in 'Description'
df['Description'][0]

'<html xmlns:fo="http://www.w3.org/1999/XSL/Format" xmlns:msxsl="urn:schemas-microsoft-com:xslt"> <head> <META http-equiv="Content-Type" content="text/html"> <meta http-equiv="content-type" content="text/html; charset=UTF-8"> </head> <body style="margin:0px 0px 0px 0px;overflow:auto;background:#FFFFFF;"> <table style="font-family:Arial,Verdana,Times;font-size:12px;text-align:left;width:100%;border-collapse:collapse;padding:3px 3px 3px 3px"> <tr style="text-align:center;font-weight:bold;background:#9CBCE2"> <td>Bitou bush</td> </tr> <tr> <td> <table style="font-family:Arial,Verdana,Times;font-size:12px;text-align:left;width:100%;border-spacing:0px; padding:3px 3px 3px 3px"> <tr> <td>FID</td> <td>0</td> </tr> <tr bgcolor="#D4E4F3"> <td>Activity</td> <td>Infestation</td> </tr> <tr> <td>ActivityDa</td> <td>1/03/2015</td> </tr> <tr bgcolor="#D4E4F3"> <td>ComName</td> <td>Bitou bush</td> </tr> <tr> <td>MonYr</td> <td>2015-03</td> </tr> <tr bgcolor="#D4E4F3"> <td>FinYr</td> <td>2014 - 2015</t

In [ ]:
# create new df for the parsing to append to

desc_df = gpd.GeoDataFrame({ 
 'FID':[''],
 'Activity':[''],
 'ActivityDa':[''],
 'ComName':[''],
 'MonYr':[''],
 'FinYr':[''],
 'Detection1':[''],
 'Date_S':['']
})

In [ ]:
# this loop will interate to parse all in descriptions, take the usefull information and append to desc_df

for string in df['Description']:
    
    s = pd.read_html(string)   #read string
    s = s[1]                   #take just the html table (not the other  )
    s = s.T                    #transpose
    s.columns = s.iloc[0]      #assign the first row as the colum headers
    
    desc_df = desc_df.append(s.iloc[1])  #append only the values (keys are already provided above)
    

In [ ]:
#a little more formatting and join back to original df

desc_df = desc_df.drop(0)                  #not sure why the first row was blank
desc_df.index = df.index                   #harmonaise the indexes
desc_df = desc_df.drop(columns=['FID'])   

df = pd.concat([df, desc_df], axis=1)

df = df.drop(columns=['Description', 'Name', 'ComName'])



In [ ]:
#these will be needed for folium
df['lon'] = df['geometry'].x
df['lat'] = df['geometry'].y

In [ ]:
df

,geometry,Activity,ActivityDa,MonYr,FinYr,Detection1,Date_S,lon,lat
0,POINT Z (153.13481 -26.71795 0.00000),Infestation,1/03/2015,2015-03,2014 - 2015,<Null>,Jan15_Jun15,153.134807,-26.717955
1,POINT Z (153.43067 -27.31563 0.00000),Infestation,25/03/2015,2015-03,2014 - 2015,<Null>,Jan15_Jun15,153.430668,-27.315630
2,POINT Z (153.42508 -27.30358 0.00000),Infestation,25/03/2015,2015-03,2014 - 2015,<Null>,Jan15_Jun15,153.425080,-27.303577
3,POINT Z (153.11057 -25.56516 0.00000),Infestation,1/05/2015,2015-05,2014 - 2015,<Null>,Jan15_Jun15,153.110568,-25.565162
4,POINT Z (153.08553 -25.76300 0.00000),Infestation,29/04/2015,2015-04,2014 - 2015,<Null>,Jan15_Jun15,153.085534,-25.763001
...,...,...,...,...,...,...,...,...,...
2455,POINT Z (153.43967 -27.76958 0.00000),Survey,8/07/2020,NaN,2019 - 2020,8/07/2020,Jan20 - Jun20,153.439668,-27.769584
2456,POINT Z (153.43592 -27.78790 0.00000),Survey,8/07/2020,NaN,2019 - 2020,8/07/2020,Jan20 - Jun20,153.435918,-27.787899
2457,POINT Z (153.43988 -27.77067 0.00000),Survey,8/07/2020,NaN,2019 - 2020,8/07/2020,Jan20 - Jun20,153.439884,-27.770671
2458,POINT Z (153.44021 -27.77354 0.00000),Survey,8/07/2020,NaN,2019 - 2020,8/07/2020,Jan20 - Jun20,153.440209,-27.773540


In [ ]:
df.to_csv('bitou.csv', index_label='FID')
